In [1]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
import numpy as np
import glob
from multiprocessing import Pool
import time

image_directory = "/Users/alidali/Downloads/818027_1400106_bundle_archive/train_v2/train/*.jpg"

def get_images_and_directory(image_directory):
    print('getting images...')
    start = time.time()
    images = []
    #directory = []
    for img in glob.glob(image_directory):
        n= cv2.imread(img)
        d = f'{img[70:]}'
        images.append([n,d])
        #directory.append(d)
    end = time.time()
    print(f'Run Time: {end-start}')
    return images


def scrape_image_data_to_df(images_list):
    print('Entered Pool Succesfully...')
    total_features = []
    images = []
    directories = []
    print('Spliting Images and Directories...')
    for i, j in images_list:
        images.append(i)
        directories.append(j)
    print('Splitted successfully...')
    count = 1
    directory_index = 0
    print('getting image data....')
    for image in images:
        directory = directories[directory_index]
        img_data = pytesseract.image_to_data(image, output_type=Output.DICT)
        total_boxes = len(img_data['text'])
        box_list = []
        sequence_list = []
        for i in range(total_boxes):
            if int(img_data['conf'][i]) > 0:
                (x ,y, w, h) = (img_data['left'][i]-4, img_data['top'][i]-4, img_data['width'][i]+8,  img_data['height'][i]+8)
                box_list.append((x,y,w,h))
                try:
                    index_num = i
                    features = {'Directory': directory,
                        'Image_Count': count,
                        'X':x,
                        'Y':y,
                        'W':w,
                        'H':h, 
                        'Confidence':img_data['conf'][index_num],
                        'Text':img_data['text'][index_num]}
                    total_features.append(features)
                    count+=1
                except:
                    pass
        directory_index += 1
    
    print('getting dataframe....')
    df = pd.DataFrame(total_features)
    return df



def parallelize_dataframe(df, func, n_cores=6):
    start = time.time()
    print('starting...')
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    print('Entering pool....')
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    end = time.time()
    print(f'ending...Time: {end - start}')
    return df

In [3]:
images = get_images_and_directory(image_directory)

getting images...
Run Time: 237.49070286750793


In [4]:
df = parallelize_dataframe(images,scrape_image_data_to_df, n_cores=6)
df.to_csv('Train_processed_final.csv')

starting...
Entering pool....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
Entered Pool Succesfully...
Spliting Images and Directories...
Splitted successfully...
getting image data....
getting dataframe....
getting dataframe....
getting dataframe....
getting dataframe....
getting dataframe....
getting dataframe....
ending...Time: 26209.768639087677


In [8]:
ground_truth = pd.read_csv('/Users/alidali/Downloads/818027_1400106_bundle_archive/written_name_train_v2.csv')

In [38]:
masters = pd.merge(df, ground_truth,how = 'left', left_on = 'Directory', right_on='FILENAME', sort= True) 

In [40]:
masters = masters.drop(['FILENAME'], axis = 1)

In [43]:
masters['Text'] = masters['Text'].str.upper()

In [44]:
correct_df = masters[masters['Text']== masters['IDENTITY']]

In [52]:
correct_df.to_csv('HW_dt.csv')

In [48]:
incorrect_df = masters[masters['Directory'].isin(correct_df['Directory'])]

In [50]:
incorrect_df = incorrect_df[incorrect_df['Text']!= incorrect_df['IDENTITY']]

In [54]:
incorrect_df.to_csv('NonHW_dt.csv')

In [ ]:
# 1. From HW_dt and NonHW_dt, get the subimages using the coordinates.
# 2. DO it before splitting the images
# 3. 

def scrape_image_data_to_df(images_list):
    print('Entered Pool Succesfully...')
    total_features = []
    images = []
    directories = []
    print('Spliting Images and Directories...')
    for i, j in images_list:
        images.append(i)
        directories.append(j)
    print('Splitted successfully...')
    count = 1
    directory_index = 0
    print('getting image data....')
    for image in images:
        directory = directories[directory_index]
        img_data = pytesseract.image_to_data(image, output_type=Output.DICT)
        total_boxes = len(img_data['text'])
        box_list = []
        sequence_list = []
        for i in range(total_boxes):
            if int(img_data['conf'][i]) > 0:
                (x ,y, w, h) = (img_data['left'][i]-4, img_data['top'][i]-4, img_data['width'][i]+8,  img_data['height'][i]+8)
                box_list.append((x,y,w,h))
                try:
                    index_num = i
                    features = {'Directory': directory,
                        'Image_Count': count,
                        'X':x,
                        'Y':y,
                        'W':w,
                        'H':h, 
                        'Confidence':img_data['conf'][index_num],
                        'Text':img_data['text'][index_num]}
                    total_features.append(features)
                    count+=1
                except:
                    pass
        directory_index += 1
    
    print('getting dataframe....')
    df = pd.DataFrame(total_features)
    return df